In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import quandl
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from pandas.tseries.offsets import *
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import itertools
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

### import data

In [42]:
#import dataset
df_features = pd.read_csv('df_features.csv', index_col = [0])
df_target = pd.read_csv('target_q.csv', index_col = [0])

In [43]:
#set dataset
df_target.index = pd.to_datetime(df_target.index)
df_features.index = pd.to_datetime(df_features.index)


In [44]:
#choose target value for the target vector
df_target = df_target['Mkt_State']

### Useful functions

In [46]:
#normalize the features dataframe
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df_features) 
df_features.loc[:,:] = scaled_values

In [15]:
#get the same index as the features vector for the target
def set_index(features, target):
    
    #remove the too many index at the end
    time_delta = int((features.index[-1] - target.index[-1]).days)
    target_bis = target[:time_delta]
    
    print('delta removed: '+ str(- time_delta))
    return target_bis

In [45]:
df_target = set_index(df_features, df_target)

delta removed: 7


In [94]:
def forward_cv(train, target):
    n_train = 500
    n_records = train.shape[0] - 10
    clf = LogisticRegression()
    cv_score = 0
    for i in range(n_train, n_records):
        train, cv = train[0:i], train[i:i+1]
        clf.fit(train, target[0:i])
        cv_score = cv_score + accuracy_score(clf.predict(cv), target[i:i+1])
        print("train:",str(train.shape[0]), "test:",str(cv.shape[0]),"cv_score:",str(cv_score))
    return(clf.param_, cv_score)


In [47]:
# create train and test sets
test = df_features.iloc[-10:]
X_train = df_features[:-10]
test_label = df_target[-10:]
Y_train = df_target[:-10]

In [33]:
training = df_features.values.reshape((df_features.shape[0],1, df_features.shape[1]))
testing = test.values.reshape((test.shape[0],1, test.shape[1]))

In [60]:
n_features = X_train.shape[1]
n_samples  = X_train.shape[0]
timesteps = 1
model = Sequential()
model.add(LSTM(n_samples, input_shape = (timesteps,n_features)))
model.add(Dense(timesteps))
model.add(Activation('sigmoid'))

In [61]:
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
history = model.fit(training, Y_train.values, epochs = 1000, batch_size = len(training), verbose = 2)

Epoch 1/1000
 - 3s - loss: 0.6770 - acc: 0.5738
Epoch 2/1000
 - 3s - loss: 0.6769 - acc: 0.5722
Epoch 3/1000
 - 3s - loss: 0.6768 - acc: 0.5733
Epoch 4/1000
 - 3s - loss: 0.6767 - acc: 0.5738
Epoch 5/1000
 - 4s - loss: 0.6766 - acc: 0.5744
Epoch 6/1000
 - 3s - loss: 0.6766 - acc: 0.5722
Epoch 7/1000
 - 4s - loss: 0.6765 - acc: 0.5722
Epoch 8/1000
 - 4s - loss: 0.6764 - acc: 0.5722
Epoch 9/1000
 - 4s - loss: 0.6763 - acc: 0.5722
Epoch 10/1000
 - 4s - loss: 0.6762 - acc: 0.5717
Epoch 11/1000
 - 5s - loss: 0.6761 - acc: 0.5706
Epoch 12/1000
 - 5s - loss: 0.6760 - acc: 0.5706
Epoch 13/1000
 - 5s - loss: 0.6760 - acc: 0.5700
Epoch 14/1000
 - 4s - loss: 0.6759 - acc: 0.5700
Epoch 15/1000
 - 4s - loss: 0.6758 - acc: 0.5679
Epoch 16/1000
 - 5s - loss: 0.6757 - acc: 0.5684
Epoch 17/1000
 - 5s - loss: 0.6757 - acc: 0.5679
Epoch 18/1000
 - 4s - loss: 0.6756 - acc: 0.5679
Epoch 19/1000
 - 4s - loss: 0.6755 - acc: 0.5679
Epoch 20/1000
 - 4s - loss: 0.6755 - acc: 0.5668
Epoch 21/1000
 - 5s - loss: 0

In [63]:
loss = model.evaluate(training, Y_train.values, verbose = 2)

In [66]:
predictions = model.predict(testing, verbose = 2)

In [69]:
preds = predictions.round()

In [70]:
preds

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.]], dtype=float32)

In [74]:
accuracy_score(preds, test_label)

0.5